In [1]:
YOUR_NAME = 'sara'

AWS_PROFILE = 'cities'

'''
# List of cities to process
cities = ["Belo Horizonte", "Campinas"]#, "Bogota", "Nairobi", "Bamako", 
        #"Lagos", "Accra", "Abidjan", "Mogadishu", "Cape Town", 
        #"Maputo", "Luanda"]

test_cities = ["Belo Horizonte"]
#cities = test_cities

cities = [city.replace(' ', '_') for city in cities]

search_buffer_files = fs.ls(SEARCH_BUFFER_PATH)

cities 

number_of_cities = len(cities)

print(f'City count: {number_of_cities}')
'''

'\n# List of cities to process\ncities = ["Belo Horizonte", "Campinas"]#, "Bogota", "Nairobi", "Bamako", \n        #"Lagos", "Accra", "Abidjan", "Mogadishu", "Cape Town", \n        #"Maputo", "Luanda"]\n\ntest_cities = ["Belo Horizonte"]\n#cities = test_cities\n\ncities = [city.replace(\' \', \'_\') for city in cities]\n\nsearch_buffer_files = fs.ls(SEARCH_BUFFER_PATH)\n\ncities \n\nnumber_of_cities = len(cities)\n\nprint(f\'City count: {number_of_cities}\')\n'

In [2]:
%load_ext autoreload

In [3]:
MAIN_PATH = "s3://wri-cities-sandbox/identifyingLandSubdivisions/data"
INPUT_PATH = f'{MAIN_PATH}/input'
CITY_INFO_PATH = f'{INPUT_PATH}/city_info'
EXTENTS_PATH = f'{CITY_INFO_PATH}/extents'
BUILDINGS_PATH = f'{INPUT_PATH}/buildings'
ROADS_PATH = f'{INPUT_PATH}/roads'
INTERSECTIONS_PATH = f'{INPUT_PATH}/intersections'
GRIDS_PATH = f'{INPUT_PATH}/city_info/grids'
SEARCH_BUFFER_PATH = f'{INPUT_PATH}/city_info/search_buffers'
OUTPUT_PATH = f'{MAIN_PATH}/output'
OUTPUT_PATH_CSV = f'{OUTPUT_PATH}/csv'
OUTPUT_PATH_RASTER = f'{OUTPUT_PATH}/raster'
OUTPUT_PATH_PNG = f'{OUTPUT_PATH}/png'
OUTPUT_PATH_RAW = f'{OUTPUT_PATH}/raw_results'

In [4]:
# Check s3 connection using AWS_PROFILE=CitiesUserPermissionSet profile 
import boto3

session = boto3.Session(profile_name=AWS_PROFILE)
s3 = session.client('s3')

# export CitiesUserPermissionSet profile to use in the next cells
import os
os.environ['AWS_PROFILE'] = AWS_PROFILE


s3.list_buckets()

{'ResponseMetadata': {'RequestId': 'GV9VSVM4M77JJ83M',
  'HostId': 'Ij+hYXCZMUXw0PzwrCDTdX0A5Ya2de3y+Jcsx96O5S9mizbvq7QDNGOt3XJH7BJhROcaXddZ5JslbpKp0WZJszdvSVrD4//UwftzeACwF/U=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Ij+hYXCZMUXw0PzwrCDTdX0A5Ya2de3y+Jcsx96O5S9mizbvq7QDNGOt3XJH7BJhROcaXddZ5JslbpKp0WZJszdvSVrD4//UwftzeACwF/U=',
   'x-amz-request-id': 'GV9VSVM4M77JJ83M',
   'date': 'Thu, 11 Dec 2025 00:47:24 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'aft-sandbox-540362055257',
   'CreationDate': datetime.datetime(2022, 9, 13, 15, 12, 20, tzinfo=tzutc()),
   'BucketArn': 'arn:aws:s3:::aft-sandbox-540362055257'},
  {'Name': 'amplify-citiesindicatorsapi-dev-10508-deployment',
   'CreationDate': datetime.datetime(2023, 8, 30, 5, 5, 13, tzinfo=tzutc()),
   'BucketArn': 'arn:aws:s3:::amplify-citiesindicatorsapi-dev-10508-deployment'},
  {'Name': 'cities-heat',
   'Crea

In [5]:
import coiled

cluster = coiled.Cluster(
    workspace="wri-cities-data",
    name=f'ils-{YOUR_NAME}',
    region="us-west-2",
    arm=True,
    worker_vm_types="r8g.xlarge",
    spot_policy="spot",
    n_workers=10,
    package_sync_ignore=["pyspark", "pypandoc"]
)
client = cluster.get_client()

print(f"Started a new Dask client on Coiled. Dashboard is available at {client.dashboard_link}")


[2025-12-10 19:47:24,776][INFO    ][coiled] Fetching latest package priorities...
[2025-12-10 19:47:24,777][INFO    ][coiled.package_sync] Resolving your local subdivisions2 Python environment...
[2025-12-10 19:47:25,197][INFO    ][coiled.package_sync] Scanning 443 conda packages...
[2025-12-10 19:47:25,201][INFO    ][coiled.package_sync] Scanning 258 python packages...
[2025-12-10 19:47:25,610][INFO    ][coiled.software_utils] No username or password found for https://conda.anaconda.org/conda-forge
[2025-12-10 19:47:26,169][INFO    ][coiled] Running pip check...
[2025-12-10 19:47:26,615][INFO    ][coiled] Validating environment...
[2025-12-10 19:47:30,192][INFO    ][coiled] Creating wheel for ~/Documents/Identifying Land Subdivisions/identifyingLandSubdivisions...
[2025-12-10 19:47:30,278][INFO    ][coiled] Creating wheel for /opt/spark-2.2.0/python...
[2025-12-10 19:47:30,445][WARNING ][coiled.package_sync] Package - debugpy, debugpy~=1.8.16 has no install candidate for Python 3.12 l

Started a new Dask client on Coiled. Dashboard is available at https://cluster-vfpae.dask.host/Clo7sgbeJrFFUf6i/status


/opt/anaconda3/envs/subdivisions2/lib/python3.12/site-packages/distributed/client.py:1590: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | Client | Scheduler | Workers |
+---------+--------+-----------+---------+
| lz4     | 4.4.4  | 4.4.5     | None    |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


2025-12-10 20:12:30,259 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
2025-12-10 20:14:15,825 - distributed.deploy.cluster - WARNING - Failed to sync cluster info multiple times - perhaps there's a connection issue? Error:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/subdivisions2/lib/python3.12/site-packages/distributed/utils.py", line 1923, in wait_for
    return await fut
           ^^^^^^^^^
  File "/opt/anaconda3/envs/subdivisions2/lib/python3.12/site-packages/distributed/comm/tcp.py", line 547, in connect
    stream = await self.client.connect(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/subdivisions2/lib/python3.12/site-packages/tornado/tcpclient.py", line 279, in connect
    af, addr, stream = await connector.start(connect_timeout=timeout)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
asyncio.exceptions.CancelledError

The above exception was the direct cause 

In [6]:
import s3fs
import fsspec
import traceback

import os


fs = s3fs.S3FileSystem(anon=False)
search_buffer_files = fs.ls(SEARCH_BUFFER_PATH)

cities = [x.split('/')[-1] for x in search_buffer_files]
len(cities)

405

In [ ]:
from gather_data_cities import *

cities = ['Accra']#,'Abidjan', 'Maputo'] # Bogota, Belo Horizonte, Campinas, Lagos, Mogadishu, Nairobi, Bamako, Cape Town, Accra, Abidjan, Luanda, Maputo
#done: 'Accra','Belo Horizonte',

city_name = 'Accra'

search_area_path = f"{SEARCH_BUFFER_PATH}/{city_name}/{city_name}_search_buffer.geoparquet"

if not fs.exists(search_area_path):
    raise FileNotFoundError(f"❌ ERROR: File does not exist in S3: {search_area_path}")
else:
    print(f"✅ File exists in S3: {search_area_path}")

with fs.open(search_area_path, mode="rb", anon=False) as f:
    search_area = gpd.read_parquet(f)

print(f"✅ Successfully read file for {city_name}: {search_area.shape[0]} rows")

rep_point = search_area.geometry.representative_point().iloc[0]
utm_proj_city = get_utm_proj(float(rep_point.x), float(rep_point.y))
transformer = pyproj.Transformer.from_crs(pyproj.CRS('EPSG:4326'), utm_proj_city, always_xy=True)
print(f"Entering OSM command for {city_name}")
osm_command(city_name, search_area, utm_proj_city)

#run_all(cities)

'''
for city_name in cities:
    search_area_path = f"{SEARCH_BUFFER_PATH}/{city_name}/{city_name}_search_buffer.geoparquet"
    #print(f"🔍 Checking S3 Path: {search_area_path}")

    if not fs.exists(search_area_path):
        raise FileNotFoundError(f"❌ ERROR: File does not exist in S3: {search_area_path}") 
    else:
        print(f"✅ File exists in S3: {search_area_path}")

    with fs.open(search_area_path, mode="rb", anon=False) as f:
        search_area = gpd.read_parquet(f)
    rep_point = search_area.geometry.representative_point().iloc[0]
    utm_proj_city = get_utm_proj(float(rep_point.x), float(rep_point.y))
    osm_command(city_name, search_area, utm_proj_city)
'''

In [ ]:
import dask.bag as db
from gather_data_cities import *

cities = ["Accra", "Abidjan", "Bamako", "Bogota", "Belo_Horizonte", "Campinas", "Cape_Town", "Lagos",  "Nairobi", "Luanda"] #, Maputo, "Mogadishu",

bag = db.from_sequence(cities, partition_size=1)
results = bag.map(gather_data_city).compute()



In [ ]:
from pre_processing import *
cities = ["Belo_Horizonte", "Campinas", "Cape_Town", "Lagos",  "Nairobi", "Luanda"] #, Maputo, "Mogadishu", #"Accra", "Abidjan", "Bamako", "Bogota", 
for city_name in cities:
    produce_blocks(city_name,YOUR_NAME).compute()

In [ ]:
roads_Accra = gpd.read_parquet('../data/input/roads/Accra/Accra_OSM_roads.geoparquet')
roads_Abidjan = gpd.read_parquet('../data/input/roads/Abidjan/Abidjan_OSM_roads.geoparquet')

In [ ]:
roads_Abidjan.head()

In [ ]:
results = []            
search_area_bounds = search_area.bounds
bbox_str = ','.join(search_area_bounds[['minx', 'miny', 'maxx', 'maxy']].values[0].astype(str))
print(f"Entering overturemaps CLI command for {city_name}")
results.append(overturemaps_download_and_save(bbox_str, "building", os.path.join(BUILDINGS_PATH, city_name), city_name))
results = pd.DataFrame(results)

In [ ]:
blocks_neatified = gpd.read_parquet('../data/input/blocks/Nairobi_blocks_sara.geoparquet')

In [ ]:
blocks_neatified

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import from_wkb  # Shapely 2.0+

df = pd.read_parquet('../data/input/blocks/Nairobi_blocks_sara.geoparquet')         # returns WKB bytes in df["geometry"]
geom = from_wkb(df["geometry"])           # vectorized -> shapely geometries
gdf  = gpd.GeoDataFrame(df.drop(columns="geometry"), geometry=geom, crs='EPSG:32637')

In [ ]:
gdf.plot()

In [ ]:
import geopandas as gpd, pyarrow, shapely, pandas as pd

print("geopandas", gpd.__version__)   # >= 0.14.0
print("pyarrow", pyarrow.__version__) # >= 12 (>=14 ideal)
print("shapely", shapely.__version__) # >= 2.0

assert isinstance(gdf, gpd.GeoDataFrame)
assert gdf.geometry.name == "geometry"
# geometry dtype should be a GeoSeries of shapely objects:
assert gdf.geometry.iloc[0].__class__.__module__.startswith("shapely")
